In [65]:
import pandas as pd
import numpy as np
import re 
from datetime import datetime

In [32]:
market_df = pd.read_csv('market_houses.csv', index_col=[0])

In [35]:
market_df.head()

,bds,bath,areas,lot_size,year_built,central_ac,central_heat,garage,price_per_sqft,elem_score,middle_score,high_score,elem_school,middle_school,high_school,price,HOA,address,neighborhood
0,3,2.0,1060.0,3118.0,1970,0,1,2 spaces,731.0,8,8,10,Warwick Elementary School,Thornton Junior High School,American High School,775000.0,0.0,"33025 Lake Erie St, Fremont, CA 94555",Lakes and Birds
1,3,2.0,1474.0,8790.0,1976,1,1,2 spaces,678.0,8,8,10,Warwick Elementary School,Thornton Junior High School,American High School,999999.0,0.0,"3340 Hogarth Pl, Fremont, CA 94555",Northgate
2,3,3.0,1826.0,4051.0,1991,1,1,2 spaces,548.0,6,8,10,Patterson Elementary School,Thornton Junior High School,American High School,999888.0,48.0,"34935 Nova Ter, Fremont, CA 94555",Northgate
3,4,1.0,1325.0,4791.0,1954,0,1,1 space,585.0,5,5,7,John Blacow Elementary School,G. M. Walters Junior High School,John F. Kennedy High School,775000.0,0.0,"40471 Blanchard St, Fremont, CA 94538",South Sundale
4,4,3.0,2072.0,10741.0,1959,1,1,2 spaces,819.0,9,10,10,Joshua Chadbourne Elementary School,William Hopkins Junior High School,Mission San Jose High School,1698000.0,0.0,"216 Saint Phillip Ct, Fremont, CA 94539",Mission Valley


In [34]:
market_df.drop(35, inplace=True)
market_df.bds = market_df.bds.apply(lambda x: x.split(' ')[0]).astype(int)
market_df.bath = market_df.bath.apply(lambda x: x.split(' ')[0]).astype(float)
market_df.areas = market_df.areas.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
market_df.lot_size = market_df.lot_size.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
market_df.lot_size = market_df.lot_size.apply(lambda x: round(x*43560,2) if x < 100 else x)#it has acre value in it
market_df.rename(columns={'a/c': 'central_ac'}, inplace=True)
market_df.central_ac = market_df.central_ac.apply(lambda x: 1 if x == 'Central' else 0)
market_df.rename(columns={'heater': 'central_heat'}, inplace=True)
market_df.central_heat = market_df.central_heat.apply(lambda x: 1 if 'Forced air' in x else 0)
market_df.price = market_df.price.str[1:].apply(lambda x: x.replace(",", "")).astype(float)
market_df.rename(columns={'price/sqft': 'price_per_sqft'}, inplace=True)
price_38 = '$'+(market_df.price[38]/market_df.areas[38]).round(2).astype(str)
market_df.loc[38, 'price_per_sqft']= price_38
market_df.price_per_sqft=market_df.price_per_sqft.apply(lambda x: x.split('$')[1].replace(",", "")).astype(float)
market_df.HOA = market_df.HOA.apply(lambda x: 0 if x == 'No Data' or x == 'None' else float(re.findall('\d+', x)[0]))

In [36]:
school_df = pd.concat([market_df['neighborhood'], market_df['elem_school'], market_df['middle_school'], market_df['high_school']], axis=1)

In [37]:
school_df.groupby(['middle_school','high_school', 'elem_school'])['high_school'].count()

middle_school                       high_school                   elem_school                        
Centerville Junior High             Washington High School        Glenmoor Elementary School             1
                                                                  Niles Elementary School                4
                                                                  Parkmont Elementary School             2
                                                                  Vallejo Mill Elementary School         1
G. M. Walters Junior High School    John F. Kennedy High School   Brier Elementary School                2
                                                                  John Blacow Elementary School          3
                                                                  John G. Mattos Elementary School       1
John M. Horner Junior High School   Irvington High School         Fred E. Weibel Elementary School       3
                                          

#From school information we come to a conclusion that one middle school corresponding to one high school and many elementary school. so we need to find out what elementary school for the null value elementary school is.

In [38]:
market_df[market_df['elem_school']=='None'].address

5      492 Mcduff Ave, Fremont, CA 94539
8     328 Grafton Ter, Fremont, CA 94539
19      315 Yampa Way, Fremont, CA 94539
Name: address, dtype: object

#After checking 3 address on the 'www.myschoollocation.com'. We found that all 3 address has the same elementary schools, Leitch elementary school (k-2) and Warm Springs Elementary School (3-6).
Leitch elementary school (k-2) A in niche, equal 9 score in great school rating.
Warm Springs Elementary School (3-6) 9 score in great school rating, so the avg score for k-6 in warm spring is 9. 
so we can replace the None value with 9.

In [39]:
market_df['elem_school'].replace('None', 'Leitch elementary school + Warm Springs Elementary School', inplace=True)
market_df['elem_score'].replace('None', 9, inplace=True)

In [40]:
market_df[['elem_score', 'middle_score', 'high_score']]=market_df[['elem_score', 'middle_score', 'high_score']].astype(int)

In [41]:
market_df.garage.value_counts()

2 spaces                      19
Attached Garage                5
3 spaces                       4
Detached Garage                3
1 space                        3
4 spaces                       2
Carport                        1
On street, Attached Garage     1
No Data                        1
Name: garage, dtype: int64

In [42]:
market_df.garage.replace('No Data', '0', inplace=True)

In [43]:
market_df.garage = market_df.garage.apply(lambda x: 2 if len(re.findall('\d', x))==0 else int(re.findall('\d', x)[0])) 
#replace attached and detached garage with park spaces 2 spaces(most frequent).

In [44]:
market_df.to_csv('market_house_cleaned.csv')

In [47]:
clean_market = pd.read_csv('market_house_cleaned.csv', index_col=[0])

In [48]:
clean_market.head()

,bds,bath,areas,lot_size,year_built,central_ac,central_heat,garage,price_per_sqft,elem_score,middle_score,high_score,elem_school,middle_school,high_school,price,HOA,address,neighborhood
0,3,2.0,1060.0,3118.0,1970,0,1,2,731.0,8,8,10,Warwick Elementary School,Thornton Junior High School,American High School,775000.0,0.0,"33025 Lake Erie St, Fremont, CA 94555",Lakes and Birds
1,3,2.0,1474.0,8790.0,1976,1,1,2,678.0,8,8,10,Warwick Elementary School,Thornton Junior High School,American High School,999999.0,0.0,"3340 Hogarth Pl, Fremont, CA 94555",Northgate
2,3,3.0,1826.0,4051.0,1991,1,1,2,548.0,6,8,10,Patterson Elementary School,Thornton Junior High School,American High School,999888.0,48.0,"34935 Nova Ter, Fremont, CA 94555",Northgate
3,4,1.0,1325.0,4791.0,1954,0,1,1,585.0,5,5,7,John Blacow Elementary School,G. M. Walters Junior High School,John F. Kennedy High School,775000.0,0.0,"40471 Blanchard St, Fremont, CA 94538",South Sundale
4,4,3.0,2072.0,10741.0,1959,1,1,2,819.0,9,10,10,Joshua Chadbourne Elementary School,William Hopkins Junior High School,Mission San Jose High School,1698000.0,0.0,"216 Saint Phillip Ct, Fremont, CA 94539",Mission Valley


In [49]:
sold_df = pd.read_csv('sold_houses.csv', index_col=[0])

In [51]:
sold_df.head()

,bds,bath,areas,lot_size,year_built,a/c,heater,garage,elem_score,middle_score,high_score,elem_school,middle_school,high_school,sold_price,HOA,address,date_sold,neighborhood
0,4 bds,2 ba,"1,942 sqft","6,960 sqft",1971,Wall,Forced air,2 spaces,7,8,10,Brookvale Elementary School,Thornton Junior High School,American High School,$1.29M,No Data,"35476 Morley Pl, Fremont, CA 94536",02/28/2020,Brookvale
1,3 bds,2 ba,"1,148 sqft","7,701 sqft",1960,Central,Forced air,2 spaces,8,5,7,Joseph Azevada Elementary School,G. M. Walters Junior High School,John F. Kennedy High School,$1.03M,No Data,"4981 Boone Dr, Fremont, CA 94538",02/28/2020,Sundale
2,4 bds,2 ba,"1,424 sqft","6,041 sqft",1984,Central,Forced air,2 spaces,8,7,8,Niles Elementary School,Centerville Junior High,Washington High School,$1.12M,No Data,"36791 Nichols Ave, Fremont, CA 94536",02/28/2020,Niles
3,2 bds,2 ba,"1,191 sqft","4,085 sqft",1978,Central,Gas,Attached Garage,8,8,10,Ardenwood Elementary School,Thornton Junior High School,American High School,"$965,000",No Data,"32840 Bluebird Loop, Fremont, CA 94555",02/28/2020,Lakes and Birds
4,4 bds,3 ba,"1,929 sqft","3,744 sqft",1991,No Data,No Data,"On street, Attached Garage",8,8,10,Ardenwood Elementary School,Thornton Junior High School,American High School,$1.39M,$37/month,"5725 Commerce Dr, Fremont, CA 94555",02/28/2020,Ardenwood


In [52]:
sold_df[sold_df.bds == 'Studio']

,bds,bath,areas,lot_size,year_built,a/c,heater,garage,elem_score,middle_score,high_score,elem_school,middle_school,high_school,sold_price,HOA,address,date_sold,neighborhood
45,Studio,-- ba,-- sqft,0.3 acres,1965,No Data,Forced air,3 spaces,6,8,10,Patterson Elementary School,Thornton Junior High School,American High School,$1.28M,No Data,"368 Staircase Falls, Fremont, CA 94539",02/12/2020,Cabrillo
60,Studio,2 ba,"2,100 sqft","9,191 sqft",1978,Wall,Forced air,Attached Garage,7,10,10,Mission Valley Elementary School,William Hopkins Junior High School,Mission San Jose High School,$1.08M,No Data,"2100 Peralta, Fremont, CA 94536",01/31/2020,Cameron Hills


In [53]:
# not enough data from row 45, so drop it
sold_df.drop(sold_df.index[45], inplace=True)
sold_df.bds = sold_df.bds.apply(lambda x: 0 if x == 'Studio' else x.split(' ')[0]).astype(int)
#after checked this house, we found the bath number is 3, so replace the none value with 3
sold_df.bath.replace('-- ba', '3 ba', inplace=True)
sold_df.bath = sold_df.bath.apply(lambda x: x.split(' ')[0]).astype(float)
sold_df.areas = sold_df.areas.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
#too many null value in this row, so we should just delete it
sold_df.drop(sold_df[sold_df.lot_size=='None'].index, inplace=True)

In [54]:
sold_df[sold_df.lot_size=='No Data']

,bds,bath,areas,lot_size,year_built,a/c,heater,garage,elem_score,middle_score,high_score,elem_school,middle_school,high_school,sold_price,HOA,address,date_sold,neighborhood
44,3,2.0,1148.0,No Data,No Data,No Data,No Data,No Data,9,10,10,Mission San Jose Elementary School,William Hopkins Junior High School,Mission San Jose High School,$1.15M,No Data,"41531 Erma Ave, Fremont, CA 94539",02/13/2020,Mission Hills
56,4,2.0,2175.0,No Data,1981,No Data,No Data,No Data,7,7,8,Tom Maloney Elementary School,Centerville Junior High,Washington High School,$1.56M,No Data,"4118 Norris Rd, Fremont, CA 94536",02/04/2020,Centerville
59,4,3.0,1785.0,No Data,No Data,No Data,No Data,No Data,9,7,8,Parkmont Elementary School,Centerville Junior High,Washington High School,$1.20M,No Data,"1145 Clay Ct, Fremont, CA 94536",02/03/2020,Parkmont


In [55]:
sold_df.iloc[43,[3,4,5,6,7]]= ['5061', '1955', 'None', 'Forced air','Attached Garage']
sold_df.iloc[54,[3,5,6,7]] = ['9,979', 'Central', 'Forced air', '2 spaces']
sold_df.iloc[57,[3,4,5,6,7]] = ['5,667', '1977', 'None', 'Forced air', '2 spaces']
sold_df.lot_size = sold_df.lot_size.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
sold_df.lot_size = sold_df.lot_size.apply(lambda x: round(x*43560,2) if x < 100 else x)#it has acre value in it
sold_df.rename(columns={'a/c': 'central_ac'}, inplace=True)
sold_df.central_ac = sold_df.central_ac.apply(lambda x: 1 if x == 'Central' else 0)
sold_df.rename(columns={'heater': 'central_heat'}, inplace=True)
sold_df.central_heat = sold_df.central_heat.apply(lambda x: 1 if 'Forced air' in x else 0)
sold_df.sold_price = sold_df.sold_price.str[1:].apply(lambda x: float(re.findall('(\d+[.,]\d+)M$', x)[0])*1000000 if 'M' in x else float(x.replace(',','')))


In [56]:
sold_df.HOA = sold_df.HOA.apply(lambda x: 0 if x == 'No Data' or x == 'None' else float(re.findall('\d+', x)[0]))

In [57]:
sold_df.garage.replace('No Data', '0', inplace=True)

In [58]:
sold_df.garage.value_counts()

Attached Garage                   33
2 spaces                          31
3 spaces                           5
On street, Attached Garage         3
Off street, Detached Garage        1
0                                  1
4 spaces                           1
5 spaces                           1
Carport, Off street, On street     1
Detached Garage                    1
Name: garage, dtype: int64

In [59]:
sold_df.garage = sold_df.garage.apply(lambda x: 2 if len(re.findall('\d', x))==0 else int(re.findall('\d', x)[0]))

In [62]:
sold_df.date_sold.head()

0    02/28/2020
1    02/28/2020
2    02/28/2020
3    02/28/2020
4    02/28/2020
Name: date_sold, dtype: object

In [68]:
#find out where the date time format is not correct.
for (i, x) in enumerate(sold_df.date_sold):
    try:
        i, datetime.strptime(x, '%m/%d/%Y').date()
    except:
        print(i, "An exception occurred")
        

39 An exception occurred


In [69]:
sold_df.loc[40,'date_sold']

'days'

In [70]:
sold_df.loc[40,'date_sold']='02/14/2020'

In [71]:
from datetime import datetime
sold_df.date_sold = sold_df.date_sold.apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())

In [72]:
sold_df[['elem_score', 'middle_score', 'high_score']]=sold_df[['elem_score', 'middle_score', 'high_score']].astype(int)

In [73]:
sold_df.to_csv('sold_house_cleaned.csv')